In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import time

import math
import matplotlib.colors as cx
import matplotlib.cm as cm

import scipy.io

from glob import glob
from scipy import signal, stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

pd.options.mode.chained_assignment = None
%matplotlib inline

In [ ]:
path = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies'
outpath = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\summaries'
outfile = os.path.join(outpath, 'snowfly_data_final.parquet')
savepath = r'G:\My Drive\Tuthill Lab Shared\Katie\presentations\lab_meeting\data\2021_06_15\figures'

In [ ]:
data = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies\12.29.20\SF0050\trial1\sf_region_temps.mat'
data = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies\3.9.21\SF0109\trial1\sf_region_temps.mat'
data = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies\3.1.21\SF0106\trial1\sf_region_temps.mat'
mat = scipy.io.loadmat(data)

In [ ]:
region_data = np.array(mat['regions'])[0]
nframes = len(region_data)

In [ ]:
px = 80
n = len(region_data)
frames = np.empty((n, px, px))

start = time.time()
for j in range(n):
    m = region_data[j].shape[0]
    p = region_data[j].shape[1]
    frames[j, :m, :p] = region_data[j]

end = time.time()
print(end - start)

In [ ]:
%matplotlib inline

start = 27125
end = 27765
vmin = np.min(frames[start:end, :, :])
vmax = np.max(frames[start:end, :, :])
ixs = np.arange(start, end, 10)
    
fig = plt.figure()
plt.clf()
plt.imshow(frames[start], cmap = 'inferno', vmin = vmin, vmax = vmax)
plt.draw()
plt.show(block=False)

for ix in ixs:
    sf_region = frames[ix]
    #print(np.min(sf))
    #print(np.max(sf))
    plt.clf()
    plt.imshow(sf_region, cmap = 'inferno', vmin = vmin, vmax = vmax)
    plt.draw()
    plt.axis('off')
    plt.pause(0.01)

In [ ]:
import skimage

start = 27125
end = 27765
vmin = np.min(frames[start:end, :, :])
vmax = np.max(frames[start:end, :, :])
ixs = np.arange(start, end, 10)
    
sigma = 3

for ix in ixs:
    sf_region = frames[ix]
    fig, ax = plt.subplots(1, 1)
    blur = skimage.filters.gaussian(sf_region, sigma=sigma)
    t = skimage.filters.threshold_otsu(blur)
    mask = blur > t
    sel = np.zeros_like(sf_region)
    sel[mask] = sf_region[mask]
    plt.imshow(sel, cmap = 'inferno', vmin = vmin, vmax = vmax)
    cbar = plt.colorbar()
    cbar.set_label('temperature ($^{\circ}$C)', fontsize = 12)
    plt.axis('off')
    plt.show()

In [ ]:
start = 27125
end = 27765
vmin = np.nanmin(frames[start:end, :, :])
vmax = np.nanmax(frames[start:end, :, :])
ixs = np.arange(start, end, 1)

xs = []
ys = []

for ix in ixs:
    sf_region = frames[ix]
    xs.extend([ix]*len(sf_region.ravel()))
    ys.extend(sf_region.ravel())
    
xs = np.array(xs)
ys = np.array(ys)

In [ ]:
start = 27125
end = 27765
#start = 27000
#end = 35000
vmin = np.nanmin(frames[start:end, :, :])
vmax = np.nanmax(frames[start:end, :, :])
ixs = np.arange(start, end, 1)

npix = len(frames[0].ravel())
matrix = np.zeros((len(ixs), npix))
matrix_sorted = np.zeros((len(ixs), npix))

for j, ix in enumerate(ixs):
    sf_region = frames[ix]
    vals = sf_region.ravel()
    matrix[j, :] = vals
    matrix_sorted[j, :] = np.sort(vals)[::-1]


In [ ]:
fig = plt.figure(figsize = (5,4))
ax = plt.gca()
plt.title('SF0050_1', fontsize = 12)
plt.xlabel('frames', fontsize = 12)
plt.ylabel('pixels', fontsize = 12)
# plt.ylim([6200, 6400])

fps = 30
nframes = matrix.shape[0]
t_frames = np.arange(0, nframes, 1)
t_s = t_frames / fps
t_m = t_frames / 60

plt.imshow(matrix.T, cmap='inferno')
cb = plt.colorbar()
plt.clim(vmin, vmax)
cb.set_label('temperature ($^{\circ}$C)', fontsize = 12)

ax.set_aspect('auto')
plt.tight_layout()
plt.show()


In [ ]:
# sorted matrix
fig = plt.figure(figsize = (5,4))
ax = plt.gca()
plt.title('SF0050_1', fontsize = 12)
plt.xlabel('frames', fontsize = 12)
plt.ylabel('pixels', fontsize = 12)
plt.ylim([0, 400])

plt.imshow(matrix_sorted.T, cmap='inferno')
cb = plt.colorbar()
plt.clim(vmin, vmax)
cb.set_label('temperature ($^{\circ}$C)', fontsize = 12)

ax.set_aspect('auto')
plt.tight_layout()
plt.show()

In [ ]:
start = 15042
end = 22308
#start = 27000
#end = 35000
vmin = np.nanmin(frames[start:end, :, :])
vmax = np.nanmax(frames[start:end, :, :])
ixs = np.arange(start, end, 1)

npix = len(frames[0].ravel())
matrix = np.empty((len(ixs), npix))
matrix_sorted = np.empty((len(ixs), npix))

for j, ix in enumerate(ixs):
    sf_region = frames[ix]
    vals = sf_region.ravel()
    vals = vals[np.isfinite(vals)]
    matrix[j, :len(vals)] = vals
    matrix_sorted[j, :len(vals)] = np.sort(vals)[::-1]


In [ ]:
fig = plt.figure(figsize = (5,4))
ax = plt.gca()
# plt.title('SF0109_1', fontsize = 12)
plt.xlabel('frames', fontsize = 12)
plt.ylabel('pixels', fontsize = 12)
# plt.ylim([6200, 6400])

fps = 30
nframes = matrix.shape[0]
t_frames = np.arange(0, nframes, 1)
t_s = t_frames / fps
t_m = t_frames / 60

plt.imshow(matrix.T, cmap='inferno')
cb = plt.colorbar()
plt.clim(vmin, vmax)
cb.set_label('temperature ($^{\circ}$C)', fontsize = 12)

ax.set_aspect('auto')
plt.tight_layout()
plt.show()

# sorted matrix
fig = plt.figure(figsize = (5,4))
ax = plt.gca()
plt.title('SF0109_1', fontsize = 12)
plt.xlabel('frames', fontsize = 12)
plt.ylabel('pixels', fontsize = 12)
plt.ylim([0, 700])

plt.imshow(matrix_sorted.T, cmap='inferno')
cb = plt.colorbar()
plt.clim(vmin, vmax)
cb.set_label('temperature ($^{\circ}$C)', fontsize = 12)

ax.set_aspect('auto')
plt.tight_layout()
plt.show()